In [ ]:
import pandas as pd
import numpy as np
import reverse_geocode

In [ ]:
# Carga del archivo
df = pd.read_csv("scrubbed.csv", low_memory=False, on_bad_lines="skip")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   datetime              80332 non-null  object 
 1   city                  80332 non-null  object 
 2   state                 74535 non-null  object 
 3   country               70662 non-null  object 
 4   shape                 78400 non-null  object 
 5   duration (seconds)    80332 non-null  object 
 6   duration (hours/min)  80332 non-null  object 
 7   comments              80317 non-null  object 
 8   date posted           80332 non-null  object 
 9   latitude              80332 non-null  object 
 10  longitude             80332 non-null  float64
dtypes: float64(1), object(10)
memory usage: 6.7+ MB


In [ ]:
df.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


In [ ]:
# Elimina caracteres alfabeticos de columnas numericas
#df['latitude'] = df['latitude'].str.extract('(\d+)', expand=False)
#df['duration (seconds)'] = df['duration (seconds)'].str.extract('(\d+)', expand=False)

# Preparación del dataset
df.rename(columns={'longitude ': 'longitude'}, inplace=True)
df[['city', 'state', 'country', 'shape', 'comments']] = df[['city', 'state', 'country', 'shape', 'comments']].astype("string")
df['country'] = df['country'].str.upper()
df['comments'] = df['comments'].str.lower()
df['datetime']= pd.to_datetime(df['datetime'], errors='coerce')
df['date posted']= pd.to_datetime(df['date posted'], errors='coerce')
df[['duration (seconds)', 'latitude', 'longitude']] = df[['duration (seconds)', 'latitude', 'longitude']].apply(pd.to_numeric, errors='coerce', axis=1).dropna()
df['duration (seconds)'] = pd.to_numeric(df['duration (seconds)'], errors='coerce')
df = df.dropna(subset=['duration (seconds)', 'latitude', 'longitude'])
df['duration (seconds)'] = df['duration (seconds)'].astype("int")

In [ ]:
# Generación de códigos de paises que faltan
missing_countries = df[['latitude', 'longitude']][df['country'].isnull()]
missing_countries['assign_country'] = [reverse_geocode.search([(r['latitude'], r['longitude'])])[0]['country_code'] for i, r in missing_countries.iterrows()]
missing_countries

,latitude,longitude,assign_country
1,29.384210,-98.581082,US
18,32.364167,-64.678611,BM
29,53.970571,-111.689885,CA
35,-38.662334,178.017649,NZ
40,41.523427,-73.646795,US
...,...,...,...
80238,39.993596,-105.089706,US
80244,43.505096,-70.637968,US
80319,40.858433,-74.163755,US
80322,50.465843,22.891814,PL


In [ ]:
df['country'][df['country'].isnull()] = missing_countries['assign_country'].values

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80328 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   datetime              79634 non-null  datetime64[ns]
 1   city                  80328 non-null  string        
 2   state                 74532 non-null  string        
 3   country               80328 non-null  string        
 4   shape                 78398 non-null  string        
 5   duration (seconds)    80328 non-null  int64         
 6   duration (hours/min)  80328 non-null  object        
 7   comments              80313 non-null  string        
 8   date posted           80328 non-null  datetime64[ns]
 9   latitude              80328 non-null  float64       
 10  longitude             80328 non-null  float64       
dtypes: datetime64[ns](2), float64(2), int64(1), object(1), string(5)
memory usage: 7.4+ MB


In [ ]:
df.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,1949-10-10 20:30:00,san marcos,tx,US,cylinder,2700,45 minutes,this event took place in early fall around 194...,2004-04-27,29.883056,-97.941111
1,1949-10-10 21:00:00,lackland afb,tx,US,light,7200,1-2 hrs,1949 lackland afb&#44 tx. lights racing acros...,2005-12-16,29.384210,-98.581082
2,1955-10-10 17:00:00,chester (uk/england),<NA>,GB,circle,20,20 seconds,green/orange circular disc over chester&#44 en...,2008-01-21,53.200000,-2.916667
3,1956-10-10 21:00:00,edna,tx,US,circle,20,1/2 hour,my older brother and twin sister were leaving ...,2004-01-17,28.978333,-96.645833
4,1960-10-10 20:00:00,kaneohe,hi,US,light,900,15 minutes,as a marine 1st lt. flying an fj4b fighter/att...,2004-01-22,21.418056,-157.803611


In [ ]:
df['comments'] = df['comments'].str.replace('&quot;','')
df['comments'] = df['comments'].str.replace('&#44','')
df['comments'] = df['comments'].str.replace('.','')
df['comments'] = df['comments'].str.replace('fire ball','fireball')
df['comments'] = df['comments'].str.replace("\(\(nuforc note:",'')
df['comments'] = df['comments'].str.replace('ufo sighting','ufo')
df['comments'] = df['comments'].str.replace('possible ufo','ufo')
df['comments'] = df['comments'].str.replace("\)\)",'')
df['comments'] = df['comments'].str.replace('bright white light','white light')
df['comments'] = df['comments'].str.replace('bright light in the sky','bright light')
df['comments'] = df['comments'].str.replace('fast moving white light','fast moving light')
df['comments'] = df['comments'].str.replace('bright lights','bright light')
df['comments'] = df['comments'].str.replace('flash of light','flashing light')
df['comments'] = df['comments'].str.replace('lights in the sky','lights')

In [101]:
# Guardado en archivo
df.to_csv('clean.csv', index=False)